In [1]:
'''
If you are running locally then 
1. reboot your local machine
2. create an environment called 'colab' using anaconda prompt
if you have a gpu
conda create -n colab python tensorflow-gpu 
if not 
conda create -n colab python tensorflow
3. to install jupyter notebook
conda install jupyter notebook
4. to go to the 'colab' environment
activate colab
5. change file path to locate this notebook and then type 'jupyter notebook'

If you use colab
1. save the data file in your google drive
2. goto colab and start running the code
'''

"\nIf you are running locally then \n1. reboot your local machine\n2. create an environment called 'colab' using anaconda prompt\nif you have a gpu\nconda create -n colab python tensorflow-gpu \nif not \nconda create -n colab python tensorflow\n3. to install jupyter notebook\nconda install jupyter notebook\n4. to go to the 'colab' environment\nactivate colab\n5. change file path to locate this notebook and then type 'jupyter notebook'\n\nIf you use colab\n1. save the data file in your google drive\n2. goto colab and start running the code\n"

In [2]:
#install prominent libraries with specific versions

#!pip install tensorflow==1.15.0
#!pip install keras==2.2.4-tf
#!pip install pandas==0.25.1
#!pip install sklearn==0.21.3
#!pip install matplotlib==3.2.1
#!pip install hyperas
#!pip install hyperopt

In [3]:
from __future__ import print_function
import pandas as pd
import numpy as np
import os
import random
import tensorflow as tf
#from tensorflow import keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, CuDNNLSTM, GRU, Input, Activation, Flatten, BatchNormalization, Reshape,Bidirectional
from tensorflow.python.keras.callbacks import ModelCheckpoint, TensorBoard, History, ReduceLROnPlateau
from tensorflow.python.keras.models import Model, load_model
from tensorflow.python.keras.layers.advanced_activations import LeakyReLU
from tensorflow.python.keras import regularizers
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from tensorflow.python.keras.wrappers.scikit_learn import KerasClassifier
import matplotlib.pyplot as plt
from matplotlib import style
from tensorflow.python.keras.optimizers import SGD, RMSprop, Adam, Adadelta
from tensorflow.python.keras.utils import np_utils
from sklearn.preprocessing import *
import datetime
from tqdm import tqdm
from tensorflow.python.keras.utils import np_utils
from tensorflow.python.keras.layers import Dense, Input, LSTM, Bidirectional, Activation, Conv1D, GRU, TimeDistributed
from tensorflow.python.keras.layers import Dropout, Embedding, GlobalMaxPooling1D, MaxPooling1D, Add, Flatten, SpatialDropout1D
from tensorflow.python.keras.layers import GlobalAveragePooling1D, BatchNormalization, concatenate
from tensorflow.python.keras.layers import Reshape, merge, Concatenate, Lambda, Average
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.layers import Layer
from tensorflow.python.keras import initializers, regularizers, constraints
from tensorflow.python.keras.initializers import *
import hyperas
import hyperopt
from hyperas.distributions import choice, uniform
from hyperas import optim
from hyperopt import Trials, STATUS_OK, tpe, rand
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D

C:\Users\kenneth\Anaconda3\envs\gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\kenneth\Anaconda3\envs\gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\kenneth\Anaconda3\envs\gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\kenneth\Anaconda3\envs\gpu\lib\site-packages\tensorflow\python\f

In [4]:
#Get library versions
print("tensorflow.__version__ = ", tf.__version__)
# import tensorflow.python.keras
# print("keras.__version__ = ", tensorflow.python.keras.__version__)
import sklearn 
print("sklearn.__version__ = ", sklearn.__version__)
print("numpy.__version__ = ", np.__version__)
print("pandas.__version__ = ", pd.__version__)
import matplotlib
print("matplotlib.__version__ = ", matplotlib.__version__)

tensorflow.__version__ =  1.10.0
sklearn.__version__ =  0.21.3
numpy.__version__ =  1.17.0
pandas.__version__ =  1.0.1
matplotlib.__version__ =  3.1.3


In [5]:
#random seed to generate reproduceable results
from numpy.random import seed
seed(56)
# from tensorflow import set_random_seed
# set_random_seed(56)
random.seed(56)
os.environ['PYTHONHASHSEED']=str(1)
os.environ['TF_KERAS'] = '1'

In [6]:
# create these folders if they do not exist
def build_path(dirName):
    try:
        os.makedirs(dirName)    
        print("Directory " , dirName ,  " Created ")
    except:
        print("Directory " , dirName ,  " already exists")  

In [7]:
# to check if GPU is getting used locally.....you need to see CPU as well as GPU in the output
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13481044582615228507
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7139449242
locality {
  bus_id: 1
  links {
  }
}
incarnation: 9396741898385275843
physical_device_desc: "device: 0, name: GeForce GTX 1070, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [8]:
def data():
  i = 7 # the label target - number of days to predict from the input date
  p = 7 #Number of days for target calculated in the data set
  batch_size=512
  CLASSES = 2
  time_steps = 7
  ticker='^GSPC'
    
  # read data
  df=pd.read_csv('../^GSPC_7_days_0_return_dtw.csv', index_col = 0, parse_dates = True)
    
  #add additional rolling mean data  
  rm_window =30
  rolling_mean = []
  for a in range(2,rm_window+1):
    df[ticker+'rm_'+str(a)] = df[ticker].rolling(window=rm_window,center=False).mean()
    rolling_mean.append(ticker+'rm_'+str(a))
    
  # create label
  targets=pd.DataFrame([])
  for j in range (1, p+1):
    targets=targets.append(df[ticker+'_{}d_target'.format(j)])
    targets=targets.append(df[ticker+'_{}d'.format(j)])
  targets=targets.T
  df=df.drop(targets.columns, axis=1)
  df=df[rm_window:-i]
  targets=targets[rm_window:-i]
  y=targets['^GSPC_{}d_target'.format(i)]

  #check for NaN and remove
  df.isna().mean().sum()
  y.isna().mean().sum()
  remove_list=[]
  for i in df.isnull().any().iteritems():
    if i[1] == True:
      remove_list.append(i[0])
  df=df.drop(remove_list, axis=1)
  df.isnull().any().mean()
 
  # add percent change
  df=df.pct_change()
  df=df.replace([np.inf, -np.inf],np.nan) 
  df.fillna(0, inplace=True)
  df.isnull().any().mean()
    
  # apply preprocessing 
  x_scaler=RobustScaler()
  x = x_scaler.fit_transform(df)
  # x_pred = x_scaler.fit_transform(x_pred)
  del df
  y=y.values
    
  # apply time steps
  def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
      v = X[i:(i + time_steps)]
      Xs.append(v)
      ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)
  x, y = create_dataset(x, y, time_steps)

  # create train and test dataset
  x_train, x_test, y_train, y_test=train_test_split(x,y, train_size=0.7, random_state=54)
  x_train = x_train.astype('float32')
  x_test = x_test.astype('float32')
  #y_train = y_train.astype('float32')
  #y_test = y_test.astype('float32')
  y_train = np_utils.to_categorical(y_train, CLASSES, 
                                    #dtype='float32'
                                   )
  y_test = np_utils.to_categorical(y_test, CLASSES, 
                                   #dtype='float32'
                                  )
    
  # adjustment for batch_size
  train_start = x_train.shape[0]%batch_size
  test_start = x_test.shape[0]%batch_size
  x_train = x_train[train_start:]
  y_train = y_train[train_start:]
  x_test = x_test[test_start:]
  y_test = y_test[test_start:]

  #reshape for 2D
  x_train1 = x_train.reshape(x_train.shape[0], 1, x_train.shape[1], x_train.shape[2]) 
  x_test1 = x_test.reshape(x_test.shape[0], 1, x_test.shape[1], x_test.shape[2])  

  embed_size = 60

  return x_train, x_test, y_train, y_test, batch_size, time_steps, embed_size, x_train1, x_test1

In [9]:
def create_multi_attention_cnn2D_bidirectional_cudnnlstm_multi_input_model(x_train, x_test, y_train, y_test, batch_size, time_steps, embed_size, x_train1, x_test1):
    
    class LayerNormalization(Layer):
        def __init__(self, eps=1e-6, **kwargs):
            self.eps = eps
            super(LayerNormalization, self).__init__(**kwargs)
        def build(self, input_shape):
            self.gamma = self.add_weight(name='gamma', shape=input_shape[-1:],initializer=Ones(), trainable=True)
            self.beta = self.add_weight(name='beta', shape=input_shape[-1:],initializer=Zeros(), trainable=True)
            super(LayerNormalization, self).build(input_shape)
        def call(self, x):
            mean = K.mean(x, axis=-1, keepdims=True)
            std = K.std(x, axis=-1, keepdims=True)
            return self.gamma * (x - mean) / (std + self.eps) + self.beta
        def compute_output_shape(self, input_shape):
            return input_shape

    class ScaledDotProductAttention():
        def __init__(self, d_model, attn_dropout=0.1):
            self.temper = np.sqrt(d_model)
            self.dropout = Dropout(attn_dropout)
        def __call__(self, q, k, v, mask):
            attn = Lambda(lambda x:K.batch_dot(x[0],x[1],axes=[2,2])/self.temper)([q, k])
            if mask is not None:
                mmask = Lambda(lambda x:(-1e+10)*(1-x))(mask)
                attn = Add()([attn, mmask])
            attn = Activation('softmax')(attn)
            attn = self.dropout(attn)
            output = Lambda(lambda x:K.batch_dot(x[0], x[1]))([attn, v])
            return output, attn

    class MultiHeadAttention():
        # mode 0 - big martixes, faster; mode 1 - more clear implementation
        def __init__(self, n_head, d_model, d_k, d_v, dropout, mode=0, use_norm=True):
            self.mode = mode
            self.n_head = n_head
            self.d_k = d_k
            self.d_v = d_v
            self.dropout = dropout
            if mode == 0:
                self.qs_layer = Dense(n_head*d_k, use_bias=False)
                self.ks_layer = Dense(n_head*d_k, use_bias=False)
                self.vs_layer = Dense(n_head*d_v, use_bias=False)
            elif mode == 1:
                self.qs_layers = []
                self.ks_layers = []
                self.vs_layers = []
                for _ in range(n_head):
                    self.qs_layers.append(TimeDistributed(Dense(d_k, use_bias=False)))
                    self.ks_layers.append(TimeDistributed(Dense(d_k, use_bias=False)))
                    self.vs_layers.append(TimeDistributed(Dense(d_v, use_bias=False)))
            self.attention = ScaledDotProductAttention(d_model)
            self.layer_norm = LayerNormalization() if use_norm else None
            self.w_o = TimeDistributed(Dense(d_model))

        def __call__(self, q, k, v, mask=None):
            d_k, d_v = self.d_k, self.d_v
            n_head = self.n_head

            if self.mode == 0:
                qs = self.qs_layer(q)  # [batch_size, len_q, n_head*d_k]
                ks = self.ks_layer(k)
                vs = self.vs_layer(v)

                def reshape1(x):
                    s = tf.shape(x)   # [batch_size, len_q, n_head * d_k]
                    x = tf.reshape(x, [s[0], s[1], n_head, d_k])
                    x = tf.transpose(x, [2, 0, 1, 3])  
                    x = tf.reshape(x, [-1, s[1], d_k])  # [n_head * batch_size, len_q, d_k]
                    return x
                qs = Lambda(reshape1)(qs)
                ks = Lambda(reshape1)(ks)
                vs = Lambda(reshape1)(vs)

                if mask is not None:
                    mask = Lambda(lambda x:K.repeat_elements(x, n_head, 0))(mask)
                head, attn = self.attention(qs, ks, vs, mask=mask)  

                def reshape2(x):
                    s = tf.shape(x)   # [n_head * batch_size, len_v, d_v]
                    x = tf.reshape(x, [n_head, -1, s[1], s[2]]) 
                    x = tf.transpose(x, [1, 2, 0, 3])
                    x = tf.reshape(x, [-1, s[1], n_head*d_v])  # [batch_size, len_v, n_head * d_v]
                    return x
                head = Lambda(reshape2)(head)
            elif self.mode == 1:
                heads = []; attns = []
                for i in range(n_head):
                    qs = self.qs_layers[i](q)   
                    ks = self.ks_layers[i](k) 
                    vs = self.vs_layers[i](v) 
                    head, attn = self.attention(qs, ks, vs, mask)
                    heads.append(head); attns.append(attn)
                head = Concatenate()(heads) if n_head > 1 else heads[0]
                attn = Concatenate()(attns) if n_head > 1 else attns[0]

            outputs = self.w_o(head)
            outputs = Dropout(self.dropout)(outputs)
            if not self.layer_norm: return outputs, attn
            # outputs = Add()([outputs, q]) # sl: fix
            return self.layer_norm(outputs), attn

    class PositionwiseFeedForward():
        def __init__(self, d_hid, d_inner_hid, dropout=0.1):
            self.w_1 = Conv1D(d_inner_hid, 1, activation='relu')
            self.w_2 = Conv1D(d_hid, 1)
            self.layer_norm = LayerNormalization()
            self.dropout = Dropout(dropout)
        def __call__(self, x):
            output = self.w_1(x) 
            output = self.w_2(output)
            output = self.dropout(output)
            output = Add()([output, x])
            return self.layer_norm(output)

    class EncoderLayer():
        def __init__(self, d_model, d_inner_hid, n_head, d_k, d_v, dropout=0.1):
            self.self_att_layer = MultiHeadAttention(n_head, d_model, d_k, d_v, dropout=dropout)
            self.pos_ffn_layer  = PositionwiseFeedForward(d_model, d_inner_hid, dropout=dropout)
        def __call__(self, enc_input, mask=None):
            output, slf_attn = self.self_att_layer(enc_input, enc_input, enc_input, mask=mask)
            output = self.pos_ffn_layer(output)
            return output, slf_attn


    def GetPosEncodingMatrix(max_len, d_emb):
        pos_enc = np.array([
            [pos / np.power(10000, 2 * (j // 2) / d_emb) for j in range(d_emb)] 
            if pos != 0 else np.zeros(d_emb) 
                for pos in range(max_len)
                ])
        pos_enc[1:, 0::2] = np.sin(pos_enc[1:, 0::2]) # dim 2i
        pos_enc[1:, 1::2] = np.cos(pos_enc[1:, 1::2]) # dim 2i+1
        return pos_enc

    def GetPadMask(q, k):
        ones = K.expand_dims(K.ones_like(q, 'float32'), -1)
        mask = K.cast(K.expand_dims(K.not_equal(k, 0), 1), 'float32')
        mask = K.batch_dot(ones, mask, axes=[2,1])
        return mask

    def GetSubMask(s):
        len_s = tf.shape(s)[1]
        bs = tf.shape(s)[:1]
        mask = K.cumsum(tf.eye(len_s, batch_shape=bs), 1)
        return mask

    class Transformer():
        def __init__(self, len_limit, embedding_matrix, d_model=embed_size, \
                  d_inner_hid=512, n_head=10, d_k=64, d_v=64, layers=2, dropout=0.1, \
                  share_word_emb=False, **kwargs):
            self.name = 'Transformer'
            self.len_limit = len_limit
            self.src_loc_info = False # True # sl: fix later
            self.d_model = d_model
            self.decode_model = None
            d_emb = d_model

            pos_emb = Embedding(len_limit, d_emb, trainable=False, \
                                weights=[GetPosEncodingMatrix(len_limit, d_emb)])

            i_word_emb = Embedding(max_features, d_emb, weights=[embedding_matrix]) # Add Kaggle provided embedding here

            self.encoder = Encoder(d_model, d_inner_hid, n_head, d_k, d_v, layers, dropout, \
                                   word_emb=i_word_emb, pos_emb=pos_emb)


        def get_pos_seq(self, x):
            mask = K.cast(K.not_equal(x, 0), 'int32')
            pos = K.cumsum(K.ones_like(x, 'int32'), 1)
            return pos * mask

        
    # define two sets of inputs(it can be the same or different)
    inputA = Input(batch_shape=(batch_size, x_train1.shape[1], x_train1.shape[2], x_train1.shape[3]), name='input_1')
    inputB = Input(batch_shape=(batch_size, x_train.shape[1], x_train.shape[2]), name='input_2')

    # the first branch operates on the first input
    x1 = Conv2D(filters = {{choice([32, 64, 128])}},name='x1', kernel_size =2,padding='same', activation='relu')(inputA)
    x2 = MaxPooling2D(pool_size=(2,2), strides=(2,2), data_format="channels_first", name='x2')(x1)
    x3 = Conv2D(filters ={{choice([16, 32, 64])}}, name='x3', kernel_size =2,padding='same', activation='relu')(x2)
    x4 = MaxPooling2D(pool_size=(2,2), strides=(2,2), data_format="channels_first", name='x4')(x3)
    x5 = Flatten(name='x5')(x4)
    x6 = Dense(units = {{choice([4, 8,16,32])}},name='x6', activation='relu')(x5)
    x = Model(inputA, x6, name='x')
    
    
    
    
   
    y1 = Bidirectional(CuDNNLSTM(units = {{choice([16,32, 64, 128])}}, name='x1',  kernel_regularizer=regularizers.l2(0.03), 
              bias_regularizer=regularizers.l1(0.03),
              return_sequences=True, stateful = True))(inputB)
    y2 = BatchNormalization(name='y2')(y1)
    y3 = Dropout({{uniform(0, 1)}},name='y3')(y2)
    
    
    if {{choice(['three', 'four','five'])}} == 'three':
        y4, slf_attn = MultiHeadAttention(n_head=3, d_model={{choice([16, 32, 64, 128])}}, d_k=64, d_v=64, dropout={{uniform(0, 1)}})(y3, y3, y3)
    elif 'four':
        y4, slf_attn = MultiHeadAttention(n_head=4, d_model={{choice([16, 32, 64, 128])}}, d_k=64, d_v=64, dropout={{uniform(0, 1)}})(y3, y3, y3)
    elif 'five':
        y4, slf_attn = MultiHeadAttention(n_head=5, d_model={{choice([16, 32, 64, 128])}}, d_k=64, d_v=64, dropout={{uniform(0, 1)}})(y3, y3, y3)   
    
    y5 = Bidirectional(CuDNNLSTM(units = {{choice([16, 32, 64, 128])}},name='y5',  kernel_regularizer=regularizers.l2(0.03), 
              bias_regularizer=regularizers.l1(0.03),
              return_sequences=False,stateful = True))(y4)
    y6 = BatchNormalization(name='y6')(y5)
    y7 = Dropout({{uniform(0, 1)}},name='y7')(y6)
    
    if {{choice(['dense', 'nodense'])}} == 'dense':
    
        y8 = Dense(units = {{choice([8, 16, 32, 64])}}, name='y8', kernel_regularizer=regularizers.l2(0.03), bias_regularizer=regularizers.l1(0.03),
                   activation='relu')(y7)
        y9 = Dropout({{uniform(0, 1)}}, name='x9')(y8)
    
        y10 = Dense(units = {{choice([4, 8, 16, 32])}}, activation='relu')(y9)
    else:
        y10 = Dense(units = {{choice([4, 8, 16, 32])}}, activation='relu')(y7)
        
    y = Model(inputB, y10, name='y')
    
    # combine the output of the two branches
    combined = Concatenate(name='x_y_output_combined')([x.output, y.output])
    z1 = Dropout(0.3)(combined)

    # combined outputs
    z = Dense(2, name='z', activation="softmax")(z1)

    # our model will accept the inputs of the two branches and then output buy or sell
    model = Model(inputs=[inputA, inputB], outputs=z)
    
    
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer = Adam(lr=0.001))

    model.summary()


    result = model.fit([x_train1, x_train],y_train,batch_size=batch_size,epochs=10,validation_data=([x_test1, x_test], y_test), verbose=0)
    
    validation_acc = np.amax(result.history['val_acc']) 

    print('Best validation acc of epoch:', result.history['val_acc'])
    print('Train acc of epoch:', result.history['acc'])
    return {'loss': -validation_acc, 'status': STATUS_OK, 'model': model}

In [10]:
if __name__ == '__main__':
    x_train, x_test, y_train, y_test, batch_size, _ ,_, x_train1, x_test1= data()
    print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)
    
    best_run, best_model = optim.minimize(model=create_multi_attention_cnn2D_bidirectional_cudnnlstm_multi_input_model, data=data ,algo=tpe.suggest, max_evals=5,trials=Trials(), notebook_name='5.5 Multi_Attention_CNN2D_Bidirectional_CuDNNLSTM_multi_input',rseed=1, verbose=False)
    print("Evaluation of best performing model:")
    #best_model.save("MULTI_ATTENTION_CNN2D_BIDIRECTIONAL_CUDNNLSTM_MULTI_INPUT_bestmodel.h5")
    #print(best_model.get_config())
    test_score, test_accuracy = best_model.evaluate([x_test1,x_test], y_test, batch_size=batch_size)
    print('test_score: ', test_score, ' test_accuracy: ', test_accuracy)
    print("Best performing model chosen hyper-parameters:")
    print(best_run)

(8704, 7, 2782) (8704, 2) (3584, 7, 2782) (3584, 2)
__________________________________________________________________________________________________                     
Layer (type)                    Output Shape         Param #     Connected to                                          
input_2 (InputLayer)            (512, 7, 2782)       0                                                                 
__________________________________________________________________________________________________                     
bidirectional (Bidirectional)   (512, 7, 128)        1458176     input_2[0][0]                                         
__________________________________________________________________________________________________                     
y2 (BatchNormalization)         (512, 7, 128)        512         bidirectional[0][0]                                   
__________________________________________________________________________________________________          

x_y_output_combined (Concatenat (512, 36)            0           x6[0][0]                                              
                                                                 dense_4[0][0]                                         
__________________________________________________________________________________________________                     
dropout_2 (Dropout)             (512, 36)            0           x_y_output_combined[0][0]                             
__________________________________________________________________________________________________                     
z (Dense)                       (512, 2)             74          dropout_2[0][0]                                       
Total params: 2,379,926                                                                                                
Trainable params: 2,379,414                                                                                            
Non-trainable params: 512               

__________________________________________________________________________________________________                     
y7 (Dropout)                    (512, 32)            0           y6[0][0]                                              
__________________________________________________________________________________________________                     
x4 (MaxPooling2D)               (512, 1, 1, 8)       0           x3[0][0]                                              
__________________________________________________________________________________________________                     
y8 (Dense)                      (512, 16)            528         y7[0][0]                                              
__________________________________________________________________________________________________                     
x5 (Flatten)                    (512, 8)             0           x4[0][0]                                              
________________________________________

time_distributed_2 (TimeDistrib (512, 7, 64)         12352       lambda_17[0][0]                                       
__________________________________________________________________________________________________                     
x1 (Conv2D)                     (512, 1, 7, 64)      712256      input_1[0][0]                                         
__________________________________________________________________________________________________                     
dropout_7 (Dropout)             (512, 7, 64)         0           time_distributed_2[0][0]                              
__________________________________________________________________________________________________                     
x2 (MaxPooling2D)               (512, 1, 3, 32)      0           x1[0][0]                                              
__________________________________________________________________________________________________                     
layer_normalization_2 (LayerNor (512, 7,

__________________________________________________________________________________________________                     
dropout_9 (Dropout)             (2048, 7, 7)         0           activation_3[0][0]                                    
__________________________________________________________________________________________________                     
lambda_20 (Lambda)              (2048, 7, 64)        0           dense_17[0][0]                                        
__________________________________________________________________________________________________                     
lambda_22 (Lambda)              (2048, 7, 64)        0           dropout_9[0][0]                                       
                                                                 lambda_20[0][0]                                       
__________________________________________________________________________________________________                     
lambda_23 (Lambda)              (512, 7,

__________________________________________________________________________________________________                     
dense_20 (Dense)                (512, 7, 192)        49152       y3[0][0]                                              
__________________________________________________________________________________________________                     
dense_21 (Dense)                (512, 7, 192)        49152       y3[0][0]                                              
__________________________________________________________________________________________________                     
lambda_24 (Lambda)              (1536, 7, 64)        0           dense_20[0][0]                                        
__________________________________________________________________________________________________                     
lambda_25 (Lambda)              (1536, 7, 64)        0           dense_21[0][0]                                        
________________________________________

Best validation acc of epoch:                                                                                          
[0.501953125, 0.5145089285714286, 0.51953125, 0.5164620535714286, 0.5142299107142857, 0.5301339285714286, 0.5298549107142857, 0.5248325892857143, 0.5418526785714286, 0.5396205357142857]
Train acc of epoch:                                                                                                    
[0.48586856617647056, 0.5207950367647058, 0.5166590073529411, 0.5076976102941176, 0.5170036764705882, 0.5155101102941176, 0.5396369485294118, 0.5271139705882353, 0.5452665441176471, 0.5531939338235294]
100%|████████████████████████████████████████████████████| 5/5 [01:53<00:00, 23.09s/it, best loss: -0.5613839285714286]
Evaluation of best performing model:
3584/3584 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 0s 121us/step
test_score:  14.733928952898298  test_accuracy:  0.5613839285714286
Best performing model chosen hyper-param